In [2]:
# PATH = os.path.join(os.getcwd(), "ml_algo")
# os.chdir(PATH)

'c:\\Users\\lukas\\OneDrive\\Pulpit\\data-science\\Projects\\bear_markets\\ml_algo'

In [3]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import xgboost as xgb
from dotenv import load_dotenv
from plotly.subplots import make_subplots
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from src.create_dataset import main

In [5]:
# Load model
MODEL_PATH = os.path.join(Path(os.getcwd()), "model", "xgboost.txt")
model = xgb.XGBClassifier()
model.load_model(MODEL_PATH)
model

XGBClassifier(alpha=3.768923163198001, base_score=0.5, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.8427237650025714, enable_categorical=False,
              gamma=1.562717477716257e-05, gpu_id=-1, importance_type=None,
              interaction_constraints='', lambda=0.16727326587860838,
              learning_rate=0.022094234577759216, max_delta_step=0, max_depth=7,
              min_child_weight=2.116247357089965, missing=nan,
              monotone_constraints='()', n_estimators=10000, n_jobs=1,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=123,
              subsample=0.8175919965588061, tree_method='auto',
              validate_parameters=1, ...)

In [6]:
# Load features

FEATURES_PATH = os.path.join(Path(os.getcwd()), "model", "features.txt")
features = []
with open(FEATURES_PATH, "r") as f:
    lines = f.readlines()
    for line in lines:
        features.append(line.strip("\n"))

In [7]:
# Get dataset with features

load_dotenv()
SECTOR = os.getenv("SECTOR")
HOST = os.getenv("HOST")
df = main(sector=SECTOR, host="mongodb://localhost:27017")

In [8]:
# Create pipeline to transform features and predict output

preprocessor = ColumnTransformer(
    remainder="drop", transformers=[("mm", MinMaxScaler(), features)]
)

pipe = Pipeline([("preprocessor", preprocessor)])

X = pd.DataFrame(pipe.fit_transform(df), columns=features)
ppred = model.predict_proba(X)[:, 1]

In [9]:
# Plot the prediction

import plotly.graph_objects as go
from plotly.subplots import make_subplots

dates = df.loc[:, "Date"].values
index = df.loc[:, "Index"].values

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(template="plotly_white", width=1500, height=750)
fig.add_trace(
    go.Line(x=dates, y=index, name="Index", line=dict(color="#385A72", width=1.5))
)
fig.add_trace(
    go.Line(
        x=dates,
        y=ppred,
        name="Probability",
        line=dict(color="#24E6C7", width=0.75),
    ),
    secondary_y=True,
)
fig.update_yaxes(range=[4.175, np.max(index) + 0.1], secondary_y=False, showgrid=False)
fig.update_yaxes(range=[0, 2], secondary_y=True, showgrid=True)
fig.update_xaxes(showgrid=True)
fig.layout.yaxis.update(showticklabels=False)
fig.layout.yaxis2.update(showticklabels=False)
fig.show()

c:\Users\lukas\OneDrive\Pulpit\data-science\Projects\bear_markets\env1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [15]:
from flask import Flask, render_template
import json
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

app = Flask(__name__)

@app.route('/')
def mlapp():
    # Plot the prediction
    dates = df.loc[:, "Date"].values
    index = df.loc[:, "Index"].values

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(template="plotly_white", width=1500, height=750)
    fig.add_trace(
        go.Line(x=dates, y=index, name="Index", line=dict(color="#385A72", width=1.5))
    )
    fig.add_trace(
        go.Line(
            x=dates,
            y=ppred,
            name="Probability",
            line=dict(color="#24E6C7", width=0.75),
        ),
        secondary_y=True,
    )
    fig.update_yaxes(range=[4.175, np.max(index) + 0.1], secondary_y=False, showgrid=False)
    fig.update_yaxes(range=[0, 2], secondary_y=True, showgrid=True)
    fig.update_xaxes(showgrid=True)
    fig.layout.yaxis.update(showticklabels=False)
    fig.layout.yaxis2.update(showticklabels=False)
    fig.show()